In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
chrom_file = '/Users/irffanalahi/Research/Research_update/SM/melcfdref/bestworkingref/PBLtilMel_meldiff-.7_other-.1_q0.00001_sorted4_1_singleCpG_final_assignedref.txt_uniq.txt'
DMR_file = '/Users/irffanalahi/Research/Research_update/SM/melcfdref/bestworkingref/PBLtilMel_meldiff-.7_other-.1_q0.00001_sorted4_1_singleCpG_final_record.txt'

chrom_data = pd.read_csv(chrom_file, sep='\t')
DMR_data = pd.read_csv(DMR_file, sep='\t')

df_chrom = pd.DataFrame(chrom_data)
df_DMR = pd.DataFrame(DMR_data)

DMR_nums = [5, 10,15, 20,25]
outfol = '/Users/irffanalahi/Research/Research_update/SM/melcfdref/bestworkingref/downsamples_number'

In [3]:
chrom = list(df_chrom['chrom'])
start = list(df_chrom['start'])

df_DMR_new = pd.DataFrame(columns=list(df_DMR.columns))

df_chrom

,chrom,start,end,celltype
0,chr10,68525811,68525813,[MEL_TUMOR]
1,chr10,68525860,68525862,[MEL_TUMOR]
2,chr10,68525961,68525963,[MEL_TUMOR]
3,chr10,68525969,68525971,[MEL_TUMOR]
4,chr10,68526037,68526039,[MEL_TUMOR]
...,...,...,...,...
915,chr7,43784751,43784753,[TIL]
916,chr7,43784815,43784817,[TIL]
917,chr7,43784850,43784852,[TIL]
918,chr7,43784957,43784959,[TIL]


In [4]:
for i in range(len(chrom)):
    df_out = df_DMR[(df_DMR['chrom'] == chrom[i]) & (df_DMR['start'] == start[i])]
    if len(df_out) != 1:
        print(len(df_out))
    df_DMR_new = pd.concat([df_DMR_new, df_out])

In [5]:
df_DMR_new

,DMRchr,DMRstart,DMRend,q,diff,chrom,start,end,celltype
0,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525811,68525813,[MEL_TUMOR]
1,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525860,68525862,[MEL_TUMOR]
2,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525961,68525963,[MEL_TUMOR]
3,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525969,68525971,[MEL_TUMOR]
4,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68526037,68526039,[MEL_TUMOR]
...,...,...,...,...,...,...,...,...,...
915,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784751,43784753,[TIL]
916,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784815,43784817,[TIL]
917,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784850,43784852,[TIL]
918,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784957,43784959,[TIL]


In [6]:
def generate_df(df_DMRs, num):
    df_quart = pd.DataFrame(columns=df_DMRs.columns)
    cells = np.array(df_DMRs.drop_duplicates(subset=['celltype'])['celltype'])
    
    for cell in cells:
        df_cell = df_DMR_new[df_DMR_new['celltype'] == cell]
        df_cell_unique = df_cell.drop_duplicates(subset=['DMRchr', 'DMRstart', 'DMRend'])
        if num <= len(df_cell_unique):
            df_samp = df_cell_unique.sample(n=num)
        else:
            df_samp = df_cell_unique
        DMR_chr = np.array(df_samp['DMRchr'])
        DMR_start = np.array(df_samp['DMRstart'])
        df_final = df_cell[(df_cell['DMRchr'].isin(DMR_chr)) & (df_cell['DMRstart'].isin(DMR_start))]
        df_quart = pd.concat([df_quart, df_final])
        
    return df_quart

In [7]:

os.mkdir(outfol)

for num in DMR_nums:
    df_out = generate_df(df_DMR_new, num)
    display(df_out)
    df_out.to_csv(outfol + '/DMR' + str(num) + '.txt', sep='\t', index=False)

,DMRchr,DMRstart,DMRend,q,diff,chrom,start,end,celltype
105,chr17,4229676,4230435,4.800000e-11,-0.861158,chr17,4229675,4229677,[MEL_TUMOR]
106,chr17,4229676,4230435,4.800000e-11,-0.861158,chr17,4229779,4229781,[MEL_TUMOR]
107,chr17,4229676,4230435,4.800000e-11,-0.861158,chr17,4229785,4229787,[MEL_TUMOR]
108,chr17,4229676,4230435,4.800000e-11,-0.861158,chr17,4229842,4229844,[MEL_TUMOR]
109,chr17,4229676,4230435,4.800000e-11,-0.861158,chr17,4230017,4230019,[MEL_TUMOR]
...,...,...,...,...,...,...,...,...,...
903,chr1,2259075,2259550,6.730000e-10,-0.427056,chr1,2259303,2259305,[TIL]
904,chr1,2259075,2259550,6.730000e-10,-0.427056,chr1,2259349,2259351,[TIL]
905,chr1,2259075,2259550,6.730000e-10,-0.427056,chr1,2259410,2259412,[TIL]
906,chr1,2259075,2259550,6.730000e-10,-0.427056,chr1,2259420,2259422,[TIL]


,DMRchr,DMRstart,DMRend,q,diff,chrom,start,end,celltype
0,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525811,68525813,[MEL_TUMOR]
1,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525860,68525862,[MEL_TUMOR]
2,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525961,68525963,[MEL_TUMOR]
3,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525969,68525971,[MEL_TUMOR]
4,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68526037,68526039,[MEL_TUMOR]
...,...,...,...,...,...,...,...,...,...
915,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784751,43784753,[TIL]
916,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784815,43784817,[TIL]
917,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784850,43784852,[TIL]
918,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784957,43784959,[TIL]


,DMRchr,DMRstart,DMRend,q,diff,chrom,start,end,celltype
27,chr13,41648540,41648756,1.460000e-06,-0.867650,chr13,41648539,41648541,[MEL_TUMOR]
28,chr13,41648540,41648756,1.460000e-06,-0.867650,chr13,41648630,41648632,[MEL_TUMOR]
29,chr13,41648540,41648756,1.460000e-06,-0.867650,chr13,41648636,41648638,[MEL_TUMOR]
30,chr13,41648540,41648756,1.460000e-06,-0.867650,chr13,41648754,41648756,[MEL_TUMOR]
31,chr7,37298409,37298573,4.530000e-07,-0.866256,chr7,37298408,37298410,[MEL_TUMOR]
...,...,...,...,...,...,...,...,...,...
915,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784751,43784753,[TIL]
916,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784815,43784817,[TIL]
917,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784850,43784852,[TIL]
918,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784957,43784959,[TIL]


,DMRchr,DMRstart,DMRend,q,diff,chrom,start,end,celltype
0,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525811,68525813,[MEL_TUMOR]
1,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525860,68525862,[MEL_TUMOR]
2,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525961,68525963,[MEL_TUMOR]
3,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525969,68525971,[MEL_TUMOR]
4,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68526037,68526039,[MEL_TUMOR]
...,...,...,...,...,...,...,...,...,...
915,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784751,43784753,[TIL]
916,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784815,43784817,[TIL]
917,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784850,43784852,[TIL]
918,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784957,43784959,[TIL]


,DMRchr,DMRstart,DMRend,q,diff,chrom,start,end,celltype
0,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525811,68525813,[MEL_TUMOR]
1,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525860,68525862,[MEL_TUMOR]
2,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525961,68525963,[MEL_TUMOR]
3,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68525969,68525971,[MEL_TUMOR]
4,chr10,68525812,68526086,3.090000e-14,-0.900649,chr10,68526037,68526039,[MEL_TUMOR]
...,...,...,...,...,...,...,...,...,...
915,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784751,43784753,[TIL]
916,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784815,43784817,[TIL]
917,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784850,43784852,[TIL]
918,chr7,43784359,43785166,3.710000e-12,-0.421380,chr7,43784957,43784959,[TIL]
